67.8


# Download

In [3]:
# !wget -O downloaded_file.zip "https://drive.usercontent.google.com/download?id=1iHPBdBOPEagvPTHZmrN__LD49emXwReY&export=download&authuser=0&confirm=t&uuid=631d60dd-569c-4bb1-a9e8-d681f0ed3d43&at=APvzH3r4me8x_LwP3n8O7lgPo8oK%3A1733988188000"
!pip -q install umap-learn
# !unzip -q downloaded_file.zip
# !wget -O ViT_all_features_merged.csv "https://drive.usercontent.google.com/download?id=1rOpINPMO5yqXycFFbJsqURRECwMiHUmP&export=download"


43.37s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [4]:
import umap


# Just Vit direct Umap

In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import umap
import plotly.graph_objects as go

def process_features(df):
    feature_cols = [c for c in df.columns if c.startswith('feat_')]
    features = df[feature_cols].values

    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features)

    return features_scaled

def reduce_to_umap(features_scaled):
    umap_direct = umap.UMAP(n_components=2, random_state=42)
    umap_direct_result = umap_direct.fit_transform(features_scaled)

    return umap_direct_result

def create_vit_umap_plot():
    # Define consistent colors for each bbox
    color_mapping = {
        'bbox1': '#FF0000',  # Red
        'bbox2': '#00FFFF',  # Cyan
        'bbox3': '#FFA500',  # Orange
        'bbox4': '#800080',  # Purple
        'bbox5': '#808080',  # Gray
        'bbox6': '#0000FF',  # Blue
        'bbox7': '#000000'   # Black
    }

    file_path = '/Users/ali/Documents/MPG|Doc/code/ViT_all_features_merged .csv'
    df = pd.read_csv(file_path)
    features_scaled = process_features(df)
    umap_direct_result = reduce_to_umap(features_scaled)

    fig = go.Figure()

    for bbox in sorted(df['bbox_name'].unique()):
        mask = df['bbox_name'] == bbox

        fig.add_trace(
            go.Scatter(
                x=umap_direct_result[mask, 0],
                y=umap_direct_result[mask, 1],
                mode='markers',
                name=bbox,
                marker=dict(
                    size=5,
                    color=color_mapping.get(bbox, '#000000'),  # Default to black if bbox not in mapping
                    opacity=0.7
                ),
                text=df[mask]['Var1'],
                hovertemplate=(
                    'Label: %{text}<br>'
                    'x: %{x:.2f}<br>'
                    'y: %{y:.2f}<br>'
                    'bbox: ' + bbox + '<br>'
                    '<extra></extra>'
                )
            )
        )

    fig.update_layout(
        height=600,
        width=800,
        title_text="ViT - Direct UMAP",
        showlegend=True,
        xaxis_title="UMAP 1",
        yaxis_title="UMAP 2"
    )

    return fig

fig = create_vit_umap_plot()
fig.show()

fig.write_html("vit_direct_umap.html")
print("Visualization has been saved as 'vit_direct_umap.html'")


/Users/ali/anaconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Visualization has been saved as 'vit_direct_umap.html'


# without seg

In [15]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import base64
import io
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
import umap
import imageio
from collections import defaultdict

# Define color mapping for bounding boxes (if still relevant for your visualization)
color_mapping = {
    'bbox1': '#FF0000',  # Red
    'bbox2': '#00FFFF',  # Cyan
    'bbox3': '#FFA500',  # Orange
    'bbox4': '#800080',  # Purple
    'bbox5': '#808080',  # Gray
    'bbox6': '#0000FF',  # Blue
    'bbox7': '#000000'   # Black
}

def process_features(df):
    feature_cols = [c for c in df.columns if c.startswith('feat_')]
    features = df[feature_cols].values
    features_scaled = StandardScaler().fit_transform(features)
    return features_scaled

file_path = 'ViT_all_features_merged.csv'
df = pd.read_csv(file_path)

if 'id' not in df.columns:
    df['id'] = df.index

def reduce_to_umap(features_scaled):
    umap_direct = umap.UMAP(n_components=2, random_state=42)
    return umap_direct.fit_transform(features_scaled)

features_scaled = process_features(df)
umap_result = reduce_to_umap(features_scaled)

df['umap_1'] = umap_result[:, 0]
df['umap_2'] = umap_result[:, 1]

# Sample 20 points for visualization
sampled_points = df.sample(20, random_state=43).sort_values('id')

class VolumeProcessor:
    def __init__(self, raw_base_dir, bbox_name, excel_file):
        self.raw_base_dir = raw_base_dir
        self.bbox_name = bbox_name
        self.excel_file = excel_file
        self.raw_vol = None
        self.synapse_data = None

    def load_volume_data(self, max_slices=None):
        raw_dir = os.path.join(self.raw_base_dir, self.bbox_name)
        raw_files = sorted([f for f in os.listdir(raw_dir) if f.startswith('slice_') and f.endswith('.tif')])

        if max_slices:
            raw_files = raw_files[:max_slices]

        raw_file_paths = [os.path.join(raw_dir, f) for f in raw_files]
        self.raw_vol = np.stack([imageio.imread(f) for f in raw_file_paths], axis=0)

    def load_synapse_data(self):
        self.synapse_data = pd.read_excel(self.excel_file)

# Helper function to convert NumPy array to base64-encoded string
def numpy_to_base64(img_array):
    pil_image = Image.fromarray(img_array.astype('uint8'))  # Convert to PIL Image
    buffer = io.BytesIO()
    pil_image.save(buffer, format="PNG")  # Save as PNG to a buffer
    buffer.seek(0)
    img_base64 = base64.b64encode(buffer.read()).decode('utf-8')  # Encode to base64
    return f"data:image/png;base64,{img_base64}"

# Initialize a cache for VolumeProcessor instances
processor_cache = {}

# Extract raw patches for sampled points
updated_patches = []

for _, row in sampled_points.iterrows():
    bbox_name = row['bbox_name']
    excel_file = f"{bbox_name}.xlsx"  # Adjust if the naming convention differs

    # Check if processor for this bbox_name is already cached
    if bbox_name not in processor_cache:
        processor = VolumeProcessor(raw_base_dir='raw',
                                    bbox_name=bbox_name,
                                    excel_file=excel_file)
        processor.load_volume_data()
        processor.load_synapse_data()
        processor_cache[bbox_name] = processor
    else:
        processor = processor_cache[bbox_name]

    central_coord = (
        int(row['central_coord_1']),
        int(row['central_coord_2']),
        int(row['central_coord_3'])
    )

    # Define the size of the patch
    half_size = 40
    x, y, z = central_coord
    x_start, x_end = max(x - half_size, 0), min(x + half_size, processor.raw_vol.shape[2])
    y_start, y_end = max(y - half_size, 0), min(y + half_size, processor.raw_vol.shape[1])
    z_start, z_end = max(z - half_size, 0), min(z + half_size, processor.raw_vol.shape[0])

    # Extract the raw patch
    raw_patch = processor.raw_vol[z, y_start:y_end, x_start:x_end]

    # Convert grayscale patch to RGB
    patch_rgb = np.stack([raw_patch] * 3, axis=-1).astype(np.uint8)
    updated_patches.append(patch_rgb)

# Create patch grid plot with updated patches
grid_rows, grid_cols = 4, 5
patch_fig = make_subplots(
    rows=grid_rows,
    cols=grid_cols,
    subplot_titles=[f"ID: {id_}" for id_ in sampled_points['id']],
    vertical_spacing=0.05,
    horizontal_spacing=0.05
)

# Add patches to the grid
for idx, patch in enumerate(updated_patches):
    row, col = divmod(idx, grid_cols)
    patch_base64 = numpy_to_base64(patch)

    patch_fig.add_trace(
        go.Image(source=patch_base64),
        row=row + 1,
        col=col + 1
    )

patch_fig.update_layout(
    title="Patch Grid Visualization",
    height=1200,
    width=1500,
    showlegend=False
)

# Reduce font size of subplot titles for clarity
for annotation in patch_fig['layout']['annotations']:
    annotation['font'] = dict(size=13)

# Show patch grid
patch_fig.show()

# Prepare UMAP scatter plot
df['color'] = df['bbox_name'].map(color_mapping)
df['size'] = 5  # Default size for all points

# Update color and size for sampled points
sampled_points = sampled_points.copy()  # To avoid SettingWithCopyWarning
sampled_points['color'] = sampled_points['bbox_name'].map(color_mapping)
sampled_points['size'] = 10  # Bigger size for sampled points

# Create UMAP scatter plot
scatter_fig = go.Figure()

# Add all points as background
scatter_fig.add_trace(go.Scatter(
    x=df['umap_1'],
    y=df['umap_2'],
    mode='markers',
    marker=dict(color=df['color'], size=df['size'], opacity=0.7),
    name="All Points",
    legendgroup="All Points",
    showlegend=False,
    hoverinfo='skip'
))

# Add sampled points with labels
scatter_fig.add_trace(go.Scatter(
    x=sampled_points['umap_1'],
    y=sampled_points['umap_2'],
    mode='markers+text',
    marker=dict(color=sampled_points['color'], size=sampled_points['size']),
    text=[f"ID: {id_}" for id_ in sampled_points['id']],
    textposition='top center',
    name="Sampled Points",
    legendgroup="Sampled Points",
    showlegend=False,
    hoverinfo='text',
    hovertext=[f"ID: {id_}, BBox: {bbox}" for id_, bbox in zip(sampled_points['id'], sampled_points['bbox_name'])]
))

# Add custom legend entries for bbox colors
for bbox, color in color_mapping.items():
    scatter_fig.add_trace(go.Scatter(
        x=[None],  # Dummy entry
        y=[None],
        mode='markers',
        marker=dict(color=color, size=6),
        name=bbox,  # Name of the bbox
        legendgroup=bbox,
        showlegend=True
    ))

# Customize layout
scatter_fig.update_layout(
    title="UMAP Scatter Plot with Color-Coded Legends",
    xaxis_title="UMAP 1",
    yaxis_title="UMAP 2",
    height=800,
    width=1000,
    legend_title="Bounding Boxes",
    legend=dict(
        itemsizing='constant'
    )
)

# Show scatter plot
scatter_fig.show()


/Users/ali/anaconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/656590314.py:66: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/656590314.py:66: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/656590314.py:66: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will swit

In [17]:
patch_fig.write_html("raw_patch_fig.html")
print("Scatter plot has been saved as 'scatter_plot.html'")

Scatter plot has been saved as 'scatter_plot.html'


# With Seg


In [10]:
import os
import glob
import numpy as np
import imageio.v2 as imageio
import pandas as pd
import matplotlib.pyplot as plt

def get_segmented_images(raw_base_dir, seg_base_dir, bbox_name, excel_file, var1, subvolume_size=80, alpha=0.1):
    """
    Extracts two 80x80 segmented 2D masks (Side 1 and Side 2) around the specified synapse
    using Var1 from the Excel file and overlays both segmentation masks on a single raw image
    with specified transparency.

    Parameters:
        raw_base_dir (str): Directory containing raw image slices.
        seg_base_dir (str): Directory containing segmentation image slices.
        bbox_name (str): Name of the bounding box to process.
        excel_file (str): Path to the Excel file containing synapse data.
        var1 (str): Var1 identifier of the synapse to process.
        subvolume_size (int): Size of the subvolume to extract around the central coordinate.
        alpha (float): Transparency level for the masks (0.0 transparent, 1.0 opaque).

    Returns:
        np.ndarray: RGB numpy array with both masks overlaid on the raw image.
    """
    # Load raw and segmentation volumes
    raw_dir = os.path.join(raw_base_dir, bbox_name)
    seg_dir = os.path.join(seg_base_dir, bbox_name)

    raw_files = sorted(glob.glob(os.path.join(raw_dir, 'slice_*.tif')))
    seg_files = sorted(glob.glob(os.path.join(seg_dir, 'slice_*.tif')))

    if len(raw_files) == 0:
        raise FileNotFoundError(f"No raw .tif files found in directory: {raw_dir}")
    if len(seg_files) == 0:
        raise FileNotFoundError(f"No segmentation .tif files found in directory: {seg_dir}")
    if len(raw_files) != len(seg_files):
        raise ValueError("Number of raw and segmentation files do not match.")

    # Load volumes
    try:
        raw_vol = np.stack([imageio.imread(f) for f in raw_files], axis=0)
        seg_vol = np.stack([imageio.imread(f).astype(np.uint32) for f in seg_files], axis=0)
    except Exception as e:
        raise IOError(f"Error loading image files: {e}")

    # Load synapse data
    try:
        synapse_data = pd.read_excel(excel_file)
    except Exception as e:
        raise IOError(f"Error reading Excel file '{excel_file}': {e}")

    # Find the synapse row based on Var1
    synapse_row = synapse_data[synapse_data['Var1'] == var1]
    if synapse_row.empty:
        raise ValueError(f"No synapse found with Var1: '{var1}' in Excel file '{excel_file}'.")

    syn_info = synapse_row.iloc[0]

    # Extract coordinates
    try:
        central_coord = (
            int(syn_info['central_coord_1']),
            int(syn_info['central_coord_2']),
            int(syn_info['central_coord_3'])
        )
        side1_coord = (
            int(syn_info['side_1_coord_1']),
            int(syn_info['side_1_coord_2']),
            int(syn_info['side_1_coord_3'])
        )
        side2_coord = (
            int(syn_info['side_2_coord_1']),
            int(syn_info['side_2_coord_2']),
            int(syn_info['side_2_coord_3'])
        )
    except KeyError as e:
        raise KeyError(f"Missing column in Excel file: {e}")
    except ValueError as e:
        raise ValueError(f"Invalid coordinate value: {e}")

    # Create segmentation masks based on coordinates
    def create_segment_masks(seg_vol, side1_coord, side2_coord):
        x1, y1, z1 = side1_coord
        x2, y2, z2 = side2_coord

        # Validate coordinates
        if not (0 <= z1 < seg_vol.shape[0] and 0 <= y1 < seg_vol.shape[1] and 0 <= x1 < seg_vol.shape[2]):
            raise ValueError("Side1 coordinates are out of bounds.")
        if not (0 <= z2 < seg_vol.shape[0] and 0 <= y2 < seg_vol.shape[1] and 0 <= x2 < seg_vol.shape[2]):
            raise ValueError("Side2 coordinates are out of bounds.")

        seg_id_1 = seg_vol[z1, y1, x1]
        seg_id_2 = seg_vol[z2, y2, x2]

        if seg_id_1 == 0:
            mask_1 = np.zeros_like(seg_vol, dtype=bool)
        else:
            mask_1 = (seg_vol == seg_id_1)

        if seg_id_2 == 0:
            mask_2 = np.zeros_like(seg_vol, dtype=bool)
        else:
            mask_2 = (seg_vol == seg_id_2)

        return mask_1, mask_2

    try:
        mask_1_full, mask_2_full = create_segment_masks(seg_vol, side1_coord, side2_coord)
    except Exception as e:
        raise ValueError(f"Error creating segmentation masks: {e}")

    # Extract subvolume around central coordinate
    half_size = subvolume_size // 2
    cx, cy, cz = central_coord

    x_start, x_end = max(cx - half_size, 0), min(cx + half_size, raw_vol.shape[2])
    y_start, y_end = max(cy - half_size, 0), min(cy + half_size, raw_vol.shape[1])
    z_slice = cz  # Assuming a single central slice in Z-axis

    if z_slice < 0 or z_slice >= raw_vol.shape[0]:
        raise ValueError("Central Z-coordinate is out of bounds.")

    sub_raw = raw_vol[z_slice, y_start:y_end, x_start:x_end]
    sub_mask_1 = mask_1_full[z_slice, y_start:y_end, x_start:x_end]
    sub_mask_2 = mask_2_full[z_slice, y_start:y_end, x_start:x_end]

    # Ensure the extracted subvolume is exactly subvolume_size x subvolume_size
    if sub_raw.shape[0] != subvolume_size or sub_raw.shape[1] != subvolume_size:
        pad_y = subvolume_size - sub_raw.shape[0]
        pad_x = subvolume_size - sub_raw.shape[1]
        sub_raw = np.pad(sub_raw, ((0, pad_y), (0, pad_x)), mode='constant', constant_values=0)
        sub_mask_1 = np.pad(sub_mask_1, ((0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
        sub_mask_2 = np.pad(sub_mask_2, ((0, pad_y), (0, pad_x)), mode='constant', constant_values=False)

    # Normalize raw image to [0, 1] for blending
    raw_normalized = sub_raw.astype(np.float32)
    raw_min, raw_max = raw_normalized.min(), raw_normalized.max()
    if raw_max != raw_min:
        raw_normalized = (raw_normalized - raw_min) / (raw_max - raw_min)
    else:
        raw_normalized = raw_normalized - raw_min  # All zeros

    raw_rgb = np.stack([raw_normalized]*3, axis=-1)  # Convert to RGB

    # Create colored masks
    side1_color = np.array([1, 0, 0])  # Red
    side2_color = np.array([0, 0, 1])  # Blue

    mask1_rgb = np.zeros_like(raw_rgb)
    mask1_rgb[sub_mask_1] = side1_color

    mask2_rgb = np.zeros_like(raw_rgb)
    mask2_rgb[sub_mask_2] = side2_color

    # Blend both masks with raw image
    overlaid_image = np.clip((1 - alpha) * raw_rgb + alpha * (mask1_rgb + mask2_rgb), 0, 1)

    # Ensure that overlapping masks don't exceed the alpha blending
    overlaid_image = np.clip(overlaid_image, 0, 1)

    # Convert back to uint8
    overlaid_image = (overlaid_image * 255).astype(np.uint8)

    return overlaid_image

def create_segmented_cube(raw_base_dir, seg_base_dir, bbox_name, excel_file, var1, subvolume_size=80, alpha=0.1):
    """
    Constructs an 80x80x80 segmented 3D cube around the specified synapse using Var1 from the Excel file
    and overlays both segmentation masks on the raw data with specified transparency for each slice.

    Parameters:
        raw_base_dir (str): Directory containing raw image slices.
        seg_base_dir (str): Directory containing segmentation image slices.
        bbox_name (str): Name of the bounding box to process.
        excel_file (str): Path to the Excel file containing synapse data.
        var1 (str): Var1 identifier of the synapse to process.
        subvolume_size (int): Size of the subvolume to extract around the central coordinate.
        alpha (float): Transparency level for the masks (0.0 transparent, 1.0 opaque).

    Returns:
        np.ndarray: RGB numpy array with both masks overlaid on each slice of the raw cube.
                   Shape: (height, width, channels, depth)
    """
    # Load raw and segmentation volumes
    raw_dir = os.path.join(raw_base_dir, bbox_name)
    seg_dir = os.path.join(seg_base_dir, bbox_name)

    raw_files = sorted(glob.glob(os.path.join(raw_dir, 'slice_*.tif')))
    seg_files = sorted(glob.glob(os.path.join(seg_dir, 'slice_*.tif')))

    if len(raw_files) == 0:
        raise FileNotFoundError(f"No raw .tif files found in directory: {raw_dir}")
    if len(seg_files) == 0:
        raise FileNotFoundError(f"No segmentation .tif files found in directory: {seg_dir}")
    if len(raw_files) != len(seg_files):
        raise ValueError("Number of raw and segmentation files do not match.")

    # Load volumes
    try:
        raw_vol = np.stack([imageio.imread(f) for f in raw_files], axis=0)
        seg_vol = np.stack([imageio.imread(f).astype(np.uint32) for f in seg_files], axis=0)
    except Exception as e:
        raise IOError(f"Error loading image files: {e}")

    # Load synapse data
    try:
        synapse_data = pd.read_excel(excel_file)
    except Exception as e:
        raise IOError(f"Error reading Excel file '{excel_file}': {e}")

    # Find the synapse row based on Var1
    synapse_row = synapse_data[synapse_data['Var1'] == var1]
    if synapse_row.empty:
        raise ValueError(f"No synapse found with Var1: '{var1}' in Excel file '{excel_file}'.")

    syn_info = synapse_row.iloc[0]

    # Extract coordinates
    try:
        central_coord = (
            int(syn_info['central_coord_1']),
            int(syn_info['central_coord_2']),
            int(syn_info['central_coord_3'])
        )
        side1_coord = (
            int(syn_info['side_1_coord_1']),
            int(syn_info['side_1_coord_2']),
            int(syn_info['side_1_coord_3'])
        )
        side2_coord = (
            int(syn_info['side_2_coord_1']),
            int(syn_info['side_2_coord_2']),
            int(syn_info['side_2_coord_3'])
        )
    except KeyError as e:
        raise KeyError(f"Missing column in Excel file: {e}")
    except ValueError as e:
        raise ValueError(f"Invalid coordinate value: {e}")

    # Create segmentation masks based on coordinates
    def create_segment_masks(seg_vol, side1_coord, side2_coord):
        x1, y1, z1 = side1_coord
        x2, y2, z2 = side2_coord

        # Validate coordinates
        if not (0 <= z1 < seg_vol.shape[0] and 0 <= y1 < seg_vol.shape[1] and 0 <= x1 < seg_vol.shape[2]):
            raise ValueError("Side1 coordinates are out of bounds.")
        if not (0 <= z2 < seg_vol.shape[0] and 0 <= y2 < seg_vol.shape[1] and 0 <= x2 < seg_vol.shape[2]):
            raise ValueError("Side2 coordinates are out of bounds.")

        seg_id_1 = seg_vol[z1, y1, x1]
        seg_id_2 = seg_vol[z2, y2, x2]

        if seg_id_1 == 0:
            mask_1 = np.zeros_like(seg_vol, dtype=bool)
        else:
            mask_1 = (seg_vol == seg_id_1)

        if seg_id_2 == 0:
            mask_2 = np.zeros_like(seg_vol, dtype=bool)
        else:
            mask_2 = (seg_vol == seg_id_2)

        return mask_1, mask_2

    try:
        mask_1_full, mask_2_full = create_segment_masks(seg_vol, side1_coord, side2_coord)
    except Exception as e:
        raise ValueError(f"Error creating segmentation masks: {e}")

    # Define the range for the cube
    half_size = subvolume_size // 2
    cx, cy, cz = central_coord

    x_start, x_end = max(cx - half_size, 0), min(cx + half_size, raw_vol.shape[2])
    y_start, y_end = max(cy - half_size, 0), min(cy + half_size, raw_vol.shape[1])
    z_start, z_end = max(cz - half_size, 0), min(cz + half_size, raw_vol.shape[0])

    sub_raw = raw_vol[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_1 = mask_1_full[z_start:z_end, y_start:y_end, x_start:x_end]
    sub_mask_2 = mask_2_full[z_start:z_end, y_start:y_end, x_start:x_end]

    # If the extracted subvolume is smaller than desired, pad it
    pad_z = subvolume_size - sub_raw.shape[0]
    pad_y = subvolume_size - sub_raw.shape[1]
    pad_x = subvolume_size - sub_raw.shape[2]

    if pad_z > 0 or pad_y > 0 or pad_x > 0:
        sub_raw = np.pad(sub_raw, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=0)
        sub_mask_1 = np.pad(sub_mask_1, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)
        sub_mask_2 = np.pad(sub_mask_2, ((0, pad_z), (0, pad_y), (0, pad_x)), mode='constant', constant_values=False)

    # Ensure the subvolume is exactly subvolume_size x subvolume_size x subvolume_size
    sub_raw = sub_raw[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_1 = sub_mask_1[:subvolume_size, :subvolume_size, :subvolume_size]
    sub_mask_2 = sub_mask_2[:subvolume_size, :subvolume_size, :subvolume_size]

    # Initialize cube for overlaid images
    overlaid_cube = np.zeros((subvolume_size, subvolume_size, 3, subvolume_size), dtype=np.uint8)

    # Define colors
    side1_color = np.array([1, 0, 0])  # Red
    side2_color = np.array([0, 0, 1])  # Blue

    # Process each slice
    for z in range(subvolume_size):
        # Normalize raw image to [0, 1] for blending
        raw_normalized = sub_raw[z].astype(np.float32)
        raw_min, raw_max = raw_normalized.min(), raw_normalized.max()
        if raw_max != raw_min:
            raw_normalized = (raw_normalized - raw_min) / (raw_max - raw_min)
        else:
            raw_normalized = raw_normalized - raw_min  # All zeros

        raw_rgb = np.stack([raw_normalized]*3, axis=-1)  # Convert to RGB

        # Create colored masks
        mask1_rgb = np.zeros_like(raw_rgb)
        mask1_rgb[sub_mask_1[z]] = side1_color

        mask2_rgb = np.zeros_like(raw_rgb)
        mask2_rgb[sub_mask_2[z]] = side2_color

        # Blend both masks with raw image
        overlaid_image = np.clip((1 - alpha) * raw_rgb + alpha * (mask1_rgb + mask2_rgb), 0, 1)

        # Ensure that overlapping masks don't exceed the alpha blending
        overlaid_image = np.clip(overlaid_image, 0, 1)

        # Convert back to uint8
        overlaid_image = (overlaid_image * 255).astype(np.uint8)

        # Assign to cube (axes: height, width, channels, depth)
        overlaid_cube[:, :, :, z] = overlaid_image

    return overlaid_cube

In [11]:
import os
import numpy as np
import pandas as pd
from PIL import Image
import base64
import io
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.preprocessing import StandardScaler
import umap
import imageio
from collections import defaultdict

# Define color mapping for bounding boxes (if still relevant for your visualization)
color_mapping = {
    'bbox1': '#FF0000',  # Red
    'bbox2': '#00FFFF',  # Cyan
    'bbox3': '#FFA500',  # Orange
    'bbox4': '#800080',  # Purple
    'bbox5': '#808080',  # Gray
    'bbox6': '#0000FF',  # Blue
    'bbox7': '#000000'   # Black
}

def process_features(df):
    feature_cols = [c for c in df.columns if c.startswith('feat_')]
    features = df[feature_cols].values
    features_scaled = StandardScaler().fit_transform(features)
    return features_scaled

file_path = 'ViT_all_features_merged.csv'
df = pd.read_csv(file_path)

if 'id' not in df.columns:
    df['id'] = df.index

def reduce_to_umap(features_scaled):
    umap_direct = umap.UMAP(n_components=2, random_state=42)
    return umap_direct.fit_transform(features_scaled)

features_scaled = process_features(df)
umap_result = reduce_to_umap(features_scaled)

df['umap_1'] = umap_result[:, 0]
df['umap_2'] = umap_result[:, 1]

# Sample 20 points for visualization
sampled_points = df.sample(20, random_state=43).sort_values('id')

# Helper function to convert NumPy array to base64-encoded string
def numpy_to_base64(img_array):
    pil_image = Image.fromarray(img_array.astype('uint8'))  # Convert to PIL Image
    buffer = io.BytesIO()
    pil_image.save(buffer, format="PNG")  # Save as PNG to a buffer
    buffer.seek(0)
    img_base64 = base64.b64encode(buffer.read()).decode('utf-8')  # Encode to base64
    return f"data:image/png;base64,{img_base64}"

# Extract raw patches for sampled points
updated_patches = []

for _, row in sampled_points.iterrows():
    bbox_name = row['bbox_name']
    excel_file = f"{bbox_name}.xlsx"  # Adjust if the naming convention differs


    seg_patch=get_segmented_images('raw','seg',bbox_name,excel_file,row['Var1'])
    # Convert grayscale patch to RGB
    patch_rgb = np.stack([raw_patch] * 3, axis=-1).astype(np.uint8)
    updated_patches.append(seg_patch)
# Create patch grid plot with updated patches
grid_rows, grid_cols = 4, 5
patch_fig = make_subplots(
    rows=grid_rows,
    cols=grid_cols,
    subplot_titles=[
        f"<b>ID:</b> {row['id']}<br><span style='font-size:10px'><b>Var1:</b> {row['Var1']}<br><b>BBox:</b> {row['bbox_name']}</span>"
        for _, row in sampled_points.iterrows()
    ],
    vertical_spacing=0.05,
    horizontal_spacing=0.05
)

# Add patches to the grid
for idx, patch in enumerate(updated_patches):
    row, col = divmod(idx, grid_cols)
    patch_base64 = numpy_to_base64(patch)

    patch_fig.add_trace(
        go.Image(source=patch_base64),
        row=row + 1,
        col=col + 1
    )

# Update layout for grid plot
patch_fig.update_layout(
    title="Patch Grid Visualization",
    height=1200,
    width=1500,
    showlegend=False
)

# Reduce font size of subplot titles for clarity
for annotation in patch_fig['layout']['annotations']:
    annotation['font'] = dict(size=10)  # Adjust font size as needed

# Show patch grid
patch_fig.show()


# Prepare UMAP scatter plot
df['color'] = df['bbox_name'].map(color_mapping)
df['size'] = 5  # Default size for all points

# Update color and size for sampled points
sampled_points = sampled_points.copy()  # To avoid SettingWithCopyWarning
sampled_points['color'] = sampled_points['bbox_name'].map(color_mapping)
sampled_points['size'] = 10  # Bigger size for sampled points

# Create UMAP scatter plot
scatter_fig = go.Figure()

# Add all points as background
scatter_fig.add_trace(go.Scatter(
    x=df['umap_1'],
    y=df['umap_2'],
    mode='markers',
    marker=dict(color=df['color'], size=df['size'], opacity=0.7),
    name="All Points",
    legendgroup="All Points",
    showlegend=False,
    hoverinfo='skip'
))

scatter_fig.add_trace(go.Scatter(
    x=sampled_points['umap_1'],
    y=sampled_points['umap_2'],
    mode='markers+text',
    marker=dict(color=sampled_points['color'], size=sampled_points['size']),
    text=[f"ID: {id_}" for id_ in sampled_points['id']],
    textposition='top center',
    name="Sampled Points",
    legendgroup="Sampled Points",
    showlegend=False,
    hoverinfo='text',
    hovertext=[f"ID: {id_}, BBox: {bbox}" for id_, bbox in zip(sampled_points['id'], sampled_points['bbox_name'])]
))

for bbox, color in color_mapping.items():
    scatter_fig.add_trace(go.Scatter(
        x=[None],  
        y=[None],
        mode='markers',
        marker=dict(color=color, size=6),
        name=bbox, 
        legendgroup=bbox,
        showlegend=True
    ))

scatter_fig.update_layout(
    title="UMAP Scatter Plot with Color-Coded Legends",
    xaxis_title="UMAP 1",
    yaxis_title="UMAP 2",
    height=800,
    width=1000,
    legend_title="Bounding Boxes",
    legend=dict(
        itemsizing='constant'
    )
)

# Show scatter plot
scatter_fig.show()


/Users/ali/anaconda3/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning:

n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.

/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:42: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:43: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:42: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will swit

In [12]:
import os
import imageio

os.makedirs("gifs", exist_ok=True)

for _, row in sampled_points.iterrows():
    bbox_name = row['bbox_name']
    excel_file = f"{bbox_name}.xlsx"
    overlaid_cube = create_segmented_cube('raw', 'seg', bbox_name, excel_file, row['Var1'])
    frames = [overlaid_cube[:, :, :, z] for z in range(overlaid_cube.shape[3])]
    gif_path = f"gifs/{row['id']}.gif"  # Correct extension
    imageio.mimsave(gif_path, frames, format='GIF', duration=0.1)

    print(f"Saved GIF for {row['id']} to {gif_path}")


/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:201: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:202: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.



Saved GIF for 17 to gifs/17.gif


/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:201: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:202: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.



Saved GIF for 103 to gifs/103.gif


/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:201: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:202: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.



...

Saved GIF for 429 to gifs/429.gif


/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:201: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:202: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.



Saved GIF for 438 to gifs/438.gif


/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:201: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.

/var/folders/c5/sm6y57p10hd9lslx7rnmkn3m0000gn/T/ipykernel_4619/477651093.py:202: DeprecationWarning:

Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.



Saved GIF for 457 to gifs/457.gif


In [13]:
# Write scatter_fig to HTML
scatter_fig.write_html("scatter_plot.html")
print("Scatter plot has been saved as 'scatter_plot.html'")

# Write patch_fig to HTML
patch_fig.write_html("patch_grid.html")
print("Patch grid has been saved as 'patch_grid.html'")

Scatter plot has been saved as 'scatter_plot.html'
Patch grid has been saved as 'patch_grid.html'
